# Performance analysis of KNN algorithm

In [ ]:
# Note on memory size needed for difference vectors
# Example: Metro dataset
n=30000 # training set size
m=10000 # test set size
D=10 # number of features
sizeof = 8 # sizeof(double)
scale = 1e9 # to scale to GB

size = n*m*D*sizeof / scale
print(f"Memory of difference vectors: {size:.1f}GB")

In [ ]:
# std
import os
import sys
import inspect
import time
import pathlib
import glob
from math import sqrt
from math import log2
# packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
%matplotlib inline

# packages
from matplotlib.colors import ListedColormap

# for selection the right path
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from common.regression_plotfunctions import *

from KNNRegressor import KNNRegressor

In [ ]:
plt.style.use("seaborn")

In [ ]:
dataset_dir_names = ["Moneyball", "Metro", "Superconductivity"]

In [ ]:
cwd = pathlib.Path(os.getcwd())
project_path = cwd.parent
data_subdir = "out/runtimes"
plot_dir = cwd / "out"


Moneyball = project_path / "Moneyball"
Metro = project_path / "Metro"
Superconductivity = project_path / "Superconductivity"

project = {
    "Moneyball": {"path": Moneyball},
    "Metro": {"path": Metro},
    "Superconductivity": {"path": Superconductivity}
}

for key, val in project.items():
    val["data_path"] = val["path"] / data_subdir
project

for name in project:
    if name not in ["Superconductivity", "Metro"]:
        continue
    data = pd.concat([pd.read_csv(file) for file in list(project[name]["data_path"].glob("*.csv"))], keys=["sklearn", "myKNN"], names=["implementation", "subindex"])
    data["total_time"] = data["train_time"] + data["inference_time"]
    project[name]["data"] = data
    print(name)
    display(data.head())

In [ ]:
def get_speedup(test):
    brutes_my = test.xs("myKNN")
    brutes_my1 = brutes_my[brutes_my["chunk_size"]==1].loc[:,["N","total_time"]]
    brutes_my2 = brutes_my[brutes_my["chunk_size"]==4].loc[:,["N","total_time"]]

    brutes_sklearn = test.xs("sklearn")
    brutes_sklearn = brutes_sklearn[brutes_sklearn["algorithm"]=="brute"]
    brutes_sklearn = brutes_sklearn.loc[:,["N","total_time"]]

    speedup = []
    for df in [brutes_sklearn, brutes_my1, brutes_my2]:
        speedup.append(df.reset_index(drop=True))

    for res in speedup:
        res["speedup"] = speedup[0]["total_time"] / res["total_time"]
    return speedup

def load_append(df, path):
    return pd.concat([df, pd.read_csv(path)])

In [ ]:
y = "inference_time"

comp_lw = 1
sns.set_palette("deep")
pal = sns.color_palette()
i = 0

def plot_myknn(y="inference_time", ax=None, save=False, i=0):
    if not ax:
        fig, ax = plt.subplots(figsize=(10,8))

    results = data.xs(implementation)

    N = results["N"].to_numpy()
    D = results["D"].unique()

    plt.sca(ax)
    #plt.plot(N, N*1e-4, ls="--", color="red", label="O(N)");
    #plt.plot(N, 1e-6*np.power(N, 2), ls="--", color="blue",label="O(N^2)");
    sns.lineplot(N, N*1e-4, ls="--", linewidth=comp_lw, label="O(N)", palette=pal[i]);
    i+=1
    sns.lineplot(N, 1e-6*np.power(N, 2), ls="--", linewidth=comp_lw, label="O(N^2)", palette=pal[i]);
    i+=1
    #plt.plot(N, np.log10(N)*1e-1, ls="--", color="green",label="O(log(N))", palette=pal[i]);
    #i+=1
    sns.lineplot(x="N", y=y, hue="chunk_size", data=results, ax=ax, palette=pal[i:i+2]);

    ax.grid(True)
    ax.set_xscale("log")
    ax.set_yscale("log")

    title = y.replace("_", " ").capitalize()
    plt.suptitle(f"{title} [{implementation}]", fontsize=36)
    ax.set_title(f"D={D}")

    if save:
        plt.savefig(plot_dir / f"{dataset_name}_{implementation}_{y}.png")

def plot_sklearnknn(y="inference_time", ax=None, save=False, i=0):
    if not ax:
        fig, ax = plt.subplots(figsize=(10,8))

    results = data.xs(implementation)
    N = results["N"].to_numpy()
    D = results["D"].unique()

    plt.sca(ax)
    #plt.plot(N, N*1e-5, ls="--", color="red",label="O(N)");
    #if y != "train_time":
    #    plt.plot(N, 1e-8*np.power(N, 2), ls="--", color="blue",label="O(N^2)");
    #plt.plot(N, np.log10(N)*1e-2, ls="--", color="green",label="O(log(N))");

    sns.lineplot(N, N*1e-5, ls="--", linewidth=comp_lw, label="O(N)", palette=pal[i]);
    i+=1
    if logN:
        sns.lineplot(N, np.log10(N)*1e-3, ls="--", linewidth=comp_lw, label="O(log(N))", palette=pal[i]);
        i+=1
    sns.lineplot(N, 1e-7*np.power(N, 2), ls="--", linewidth=comp_lw, label="O(N^2)", palette=pal[i]);
    i+=1
    
    sns.lineplot(x="N", y=y, hue="algorithm", data=results, ax=ax, palette=pal[i:i+3]);

    ax.grid(True)
    ax.set_xscale("log")
    ax.set_yscale("log")

    title = y.replace("_", " ").capitalize()
    plt.suptitle(f"{title} [{implementation}]", fontsize=36)
    ax.set_title(f"D={D}")

    if save:
        plt.savefig(plot_dir / f"{dataset_name}_{implementation}_{y}.png")

def plot_bothknn(y="inference_time", ax=None, save=False, idx=0, labels=None):
    if not ax:
        fig, ax = plt.subplots(figsize=(10,8))
    plt.sca(ax)

    N = data["N"].unique()
    D = data["D"].unique()

    i = idx
    j = i+3
    handles = []
    for imp in ["sklearn", "myKNN"]:
        hue = "algorithm" if imp=="sklearn" else "chunk_size"
        ls = "-"
        sns.lineplot(x="N", y=y, hue=hue, data=data.xs(imp), ax=ax, palette=pal[i:j]);
        if imp=="sklearn":
            i += 3
            j += 2
        else:
            i += 2
    sns.lineplot(N, N*1e-5, ls="--", linewidth=comp_lw, label="O(N)",palette=pal[i]); 
    i+=1
    if logN:
        sns.lineplot(N, np.log10(N)*1e-3, ls="--", linewidth=comp_lw, label="O(log(N))", palette=pal[i]); 
        
        i+=1
    sns.lineplot(N, 1e-7*np.power(N, 2), ls="--", linewidth=comp_lw, label="O(N^2)", palette=pal[i]); 
    
    i+=1

    if labels != None:
        handles, old_labels = ax.get_legend_handles_labels()
        new_labels = []
        for line, old_label, new_label in zip(handles, old_labels, labels):
            #print(f"replaced {old_label} with {new_label}")
            new_labels.append(new_label)
    
    ax.legend(handles, new_labels)

    ax.grid(True)
    ax.set_xscale("log")
    ax.set_yscale("log")

    title = y.replace("_", " ").capitalize()
    plt.suptitle(f"{title}", fontsize=36)
    ax.set_title(f"D={D}")

    if save:
        plt.savefig(plot_dir / f"{dataset_name}_comparison_{y}.png")

def plot_speedup(data, ax=None, save=False, i=0, logx=True, logy=True):
    if not ax:
        fig, ax = plt.subplots(figsize=(6,4))

    plt.sca(ax)
    
    handles = []
    for model, label in zip(data, ["sklearn-brute", "myKNN-1", "myKNN-4"]):
        h, _ = sns.lineplot(x="N", y="speedup", data=model, ax=ax, label=label);
        handles.append(h)
    
    ax.grid(True)
    if logx:
        ax.set_xscale("log")
    if logy:
        ax.set_yscale("log")

    title = f"Speedup [{dataset_name}]"
    plt.suptitle(f"{title}", fontsize=36)
    D = 10 if dataset_name=="Metro" else 81
    ax.set_title(f"D={D}")

    if save:
        plt.savefig(plot_dir / f"{dataset_name}_speedup.png")

def print_styling(figsize=(14,8)):

    plt.rc('figure', figsize=figsize) 
    SMALL_SIZE = 15
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 26

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('lines', linewidth=2)

    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


# Metro

In [ ]:
data = project["Metro"]["data"]
dataset_name = "Metro"
data.head()

In [ ]:
print_styling((10,8))
ax = plt.gca()
sns.set_palette("deep")

labels = [
    "sklearn(brute)", "sklearn(kd_tree)", "sklearn(ball_tree)",
    "myKNN(chunk_size=1)", "myKNN(chunk_size=4)",
    "O(N)", "O(N^2)"]
logN = False
ax.set_ylim((1e-4, 1e2))
plot_bothknn(y="total_time", ax=ax, save=True, idx=0, labels=labels)

In [ ]:
ax.get_legend_handles_labels()

In [ ]:
# Plot speedup
print_styling((10,8))
speedups = get_speedup(data)
plot_speedup(speedups, save=True, logy=True, ax=plt.gca())

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(16,8))
sns.set_palette("deep")

implementation = "sklearn"
logN = False
i = 0
plot_sklearnknn("total_time", ax=axs[0], i=i)
implementation = "myKNN"
plot_myknn("total_time", ax=axs[1], i=i)

# Superconductivity

In [ ]:
data = project["Superconductivity"]["data"]
dataset_name = "Superconductivity"
data.head()

In [ ]:
print_styling((10,8))
ax = plt.gca()
sns.set_palette("deep")

labels = [
    "sklearn(brute)", "sklearn(kd_tree)", "sklearn(ball_tree)",
    "myKNN(chunk_size=1)", "myKNN(chunk_size=4)",
    "O(N)", "O(N^2)"]
logN = False
ax.set_ylim((1e-4, 1e2))
plot_bothknn(y="total_time", ax=ax, save=True, idx=0, labels=labels)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,8))
sns.set_palette("deep")

implementation = "sklearn"
logN = False
i = 0
plot_sklearnknn("total_time", ax=axs[0], i=i)
implementation = "myKNN"
plot_myknn("total_time", ax=axs[1], i=i)

In [ ]:
# Plot speedup
print_styling((10,8))
speedups = get_speedup(data)
plot_speedup(speedups, save=True, logy=False,  ax=plt.gca())